In [1]:
import numpy as np
from bokeh.io import push_notebook, output_notebook, show
from bokeh.plotting import figure, curdoc
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, Slider, CustomJS
from bokeh.themes import Theme

output_notebook()
curdoc().theme = 'dark_minimal'

Loading BokehJS ...

# Demonstration of Anderson Localization

Anderson localization refers to the absence of diffusion of waves in a disordered medium. This phenomenon was first identified by P.W. Anderson in 1958 when he studied the electronic states in a disordered lattice. 

## Theoretical Background

In a crystal, electrons can move freely through the lattice, leading to electrical conduction. However, if there's a lot of disorder in the lattice, these electrons can become trapped, making the material an insulator. This trapping of electrons (or more generally, waves) due to disorder is known as Anderson localization.

### 1D Model System

To demonstrate Anderson localization, let's consider a simple model: a 1D chain of \(N\) atoms with periodic boundary conditions. One can visualize this system as a ring formed by \(N\) atoms.

![anderson.png](anderson.png)

The Hamiltonian with nearest-neighbour hopping integrals \( t \) is given by:

$$
\left[\begin{array}{ccllcc}
0 & t & & & & t\\
t & 0 & t\\
 & t & 0 & t\\
 &  & t & \text{……}\\
 &  &  &  & 0 & t\\
t &  &  &  & t & 0
\end{array}\right]
$$

We introduce disorder in the system by adding random on-site potentials \( d_i \). The Hamiltonian then becomes:

$$
\left[\begin{array}{ccllcc}
d_{1} & t  & & & & t\\
t & d_{2} & t\\
 & t & d_{3} & t\\
 &  & t & \text{……}\\
 &  &  &  & d_{N-1} & t\\
t &  &  &  & t & d_{N}
\end{array}\right]
$$

### Localization Phenomenon

To quantitatively understand the localization phenomenon, one can study the eigenstates of the system. Specifically, the inverse participation ratio (IPR) is a useful metric. It's defined as:

$$
IPR=\frac{1}{\Sigma p_i ^2}=\frac{1}{\Sigma |\psi _i |^4}
$$

In this equation, \( p_i \) is the probability density of the i-th state. For a state that's fully delocalized over \( N \) atoms, \( IPR = N \). However, for a completely localized state, \( IPR = 1 \).

In the provided plots, you'll observe the IPR for various eigenstates and the probability density of the \( n^{th} \) wavefunction.



In [2]:
def IPR(vector):
    v1 = abs(vector)**2
    v1 = v1**2
    ipr = 1.0 / np.sum(v1)
    return ipr

def buildham(t,nq):
    nq=nq-1
    y=np.ones(nq)
    mat1=np.diag(y,-1)*t+np.diag(y,1)*t
    mat1[nq,0]=t
    mat1[0,nq]=t 
    return mat1

ndim=50
strength=1.0
nr=np.arange(0,ndim,1)
H0=buildham(-1.0,ndim)
disorder=strength*(np.random.rand(ndim)-0.5)
H1=H0+np.diag(disorder)
[eigval0,eigvec0]=np.linalg.eigh(H0)
[eigval1,eigvec1]=np.linalg.eigh(H1)
ipa1=np.zeros(ndim)
ipa0=np.zeros(ndim)

for i in range(0,ndim-1):
    ipa1[i]=IPR(eigvec1[:,i])
    ipa0[i]=IPR(eigvec0[:,i])

source_ipr0 = ColumnDataSource(data=dict(x=nr, y=ipa0))
source_ipr1 = ColumnDataSource(data=dict(x=nr, y=ipa1))

p1 = figure(title="IPR", x_axis_label='Eigenstate index', y_axis_label='IPR', height=500, width=1000)
p1.circle('x', 'y', source=source_ipr0, color='blue', legend_label='IPR, ideal')
p1.circle('x', 'y', source=source_ipr1, color='red', legend_label='IPR, disordered')

source_wf0 = ColumnDataSource(data=dict(x=nr, y=abs(eigvec0[:,0])**2))
source_wf1 = ColumnDataSource(data=dict(x=nr, y=abs(eigvec1[:,0])**2))

p2 = figure(title="Probability Density", x_axis_label='Atom index', y_axis_label='Probability density', height=500, width=1000)
p2.line('x', 'y', source=source_wf0, color='blue', legend_label='WF, ideal')
p2.line('x', 'y', source=source_wf1, color='red', legend_label='WF, disordered')

callback = CustomJS(args=dict(source_wf0=source_wf0, source_wf1=source_wf1, eigvec0=eigvec0, eigvec1=eigvec1, nr=nr), code="""
    const n = cb_obj.value;
    const wf0 = Array.from(eigvec0).map(row => Math.pow(Math.abs(row[n]), 2));
    const wf1 = Array.from(eigvec1).map(row => Math.pow(Math.abs(row[n]), 2));
    source_wf0.data.y = wf0;
    source_wf1.data.y = wf1;
    source_wf0.change.emit();
    source_wf1.change.emit();
""")

slider = Slider(start=0, end=ndim-1, value=0, step=1, title="n")
slider.js_on_change('value', callback)

layout = column(p1, p2, slider)
show(layout)



## Questions for Exploration

1. How does localization vary with the eigenstate index, disorder strength (i.e., the range of \($ d_i $\) values), and system size \($ N $\)?
2. Why is \($ IPR = N $\) achieved only for the first (and in the case of even \($ N $\), the last) eigenstate of an ideal system without any disorder?
3. How does disorder influence the eigenvalues of the system?
4. Consider the extreme case where the disorder is so strong that each on-site potential \($ d_i $\) is very large. What would you expect the behavior of the electrons to be?
5. Relate the concept of Anderson localization to real-world materials. Can you think of any applications or phenomena where this plays a crucial role?
6. How does increasing the dimensionality (from 1D to 2D or 3D) affect Anderson localization?
7. How can external factors, like temperature or external fields, influence Anderson localization?

By exploring the above questions, you'll gain a deeper understanding of the fascinating phenomenon of Anderson localization and its implications in condensed matter physics.